In [9]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import os  # For path checking
import time  # For introducing delays

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine globally
engine = pyttsx3.init()

def give_audio_feedback(feedback):
    try:
        engine.say(feedback)
        engine.runAndWait()
    except RuntimeError as e:
        print("Error during audio feedback:", e)

def extract_landmarks(image_path, pose_model):
    if not os.path.exists(image_path):
        print("Error: File not found:", image_path)
        return None
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose_model.process(image_rgb)
    if results.pose_landmarks:
        selected_landmarks = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
        return np.array([[lm.x, lm.y, lm.z] for i, lm in enumerate(results.pose_landmarks.landmark) if i in selected_landmarks]).flatten()
    else:
        print("No pose detected in reference image.")
        return None

def compare_landmarks(predicted_landmarks, reference_landmarks, frame, tolerance=0.25):
    predicted_landmarks = predicted_landmarks.reshape(-1, 3)
    reference_landmarks = reference_landmarks.reshape(-1, 3)
    deviations = np.linalg.norm(predicted_landmarks - reference_landmarks, axis=1)
    
    feedback = ""
    if all(dev > tolerance for dev in deviations):
        feedback = "Please perform the exercise."
    elif deviations[1] > tolerance or deviations[2] > tolerance:  # Shoulders
        feedback = "Please adjust your shoulders."
    elif any(deviations[i] > tolerance for i in [3, 4, 5, 6]):  # Arms/Hands
        feedback = "Please lift or adjust your hands."
    elif any(deviations[i] > tolerance for i in [7, 8, 9, 10]):  # Legs
        feedback = "Please adjust your legs."
    elif deviations[0] > tolerance:  # Head
        feedback = "Please align your head properly."
    
    if feedback:
        cv2.putText(frame, feedback, (10, 40), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "Pose is correct!", (10, 40), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    
    return feedback

def detect_pose(frame, pose_model):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_model.process(image_rgb)
    if results.pose_landmarks:
        selected_landmarks = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
        landmarks = np.array([[lm.x, lm.y, lm.z] for i, lm in enumerate(results.pose_landmarks.landmark) if i in selected_landmarks])
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        return frame, landmarks
    return frame, None

# Load the reference pose landmarks
reference_landmarks = extract_landmarks(r'C:\Users\HP\Downloads\DATASET\DATASET\TRAIN\tree\00000075.jpg', pose_video)
if reference_landmarks is None:
    print("Error: Could not extract reference landmarks.")
    exit()

camera_video = cv2.VideoCapture(0)  # Use 0 for the default camera
if not camera_video.isOpened():
    print("Error: Unable to access camera.")
    exit()

while camera_video.isOpened():
    ok, frame = camera_video.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    
    frame, landmarks = detect_pose(frame, pose_video)
    
    if landmarks is not None:
        feedback = compare_landmarks(landmarks, reference_landmarks, frame, tolerance=0.25)
        give_audio_feedback(feedback)
    
    # Display the frame
    cv2.imshow("Yoga Pose Feedback", frame)

    # Exit on pressing the ESC key
    if cv2.waitKey(1) & 0xFF == 27:
        break

camera_video.release()
cv2.destroyAllWindows()
